In [2]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from utils import MyTabNetClassifier
from acv_explainers.utils import *
from ares import AReS
from cet import CounterfactualExplanationTree
from ce import ActionExtractor
from utils import DatasetHelper, submodular_picking, DATASETS_NAME
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

seed = 2022
np.random.seed(0)

In [3]:
# Load Compas data

seed = 2022
np.random.seed(0)
dataset = 'c'
dataset_name = DATASETS_NAME[dataset]
model= 'X'

LAMBDA = 0.01
GAMMA = 1.0

D = DatasetHelper(dataset=dataset, feature_prefix_index=False)
X_tr, X_ts, y_tr, y_ts = D.train_test_split()

In [4]:
# Train IsolationForest for OutlierScore

from sklearn.ensemble import IsolationForest

isolation = IsolationForest()
isolation.fit(X_tr)

IsolationForest()

In [5]:
y_tr = 1 - y_tr
y_ts = 1 - y_ts

In [6]:
# Train the model to explain

mdl = LGBMClassifier(n_estimators=50, num_leaves=8)
mdl.fit(X_tr, y_tr)

LGBMClassifier(n_estimators=50, num_leaves=8)

In [7]:
D.feature_names

['age',
 'juv_fel_count',
 'juv_misd_count',
 'juv_other_count',
 'priors_count',
 'race:African-American',
 'race:Asian',
 'race:Caucasian',
 'race:Hispanic',
 'race:Native American',
 'race:Other',
 'c_charge_degree:F',
 'c_charge_degree:M',
 'gender']

In [8]:
D.target_name

'RecidivateWithinTwoYears'

In [24]:
D.feature_constraints

['INC',
 '',
 '',
 '',
 '',
 'FIX',
 'FIX',
 'FIX',
 'FIX',
 'FIX',
 'FIX',
 '',
 '',
 'FIX']

In [25]:
D.feature_categories

[[5, 6, 7, 8, 9, 10], [11, 12]]

In [11]:
D.feature_types

['I', 'I', 'I', 'I', 'I', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']

In [12]:
X = X_tr[mdl.predict(X_tr)==1]
X_vl = X_ts[mdl.predict(X_ts)==1]

## Actionable Recourse Summary

In [14]:
print('## Actionable Recourse Summary')
ares = AReS(mdl, X_tr, max_rule=4, max_rule_length=4, discretization_bins=10, minimum_support=0.05, print_objective=False,
            feature_names=D.feature_names, feature_types=D.feature_types, feature_categories=D.feature_categories, 
            feature_constraints=D.feature_constraints, target_name=D.target_name, target_labels=D.target_labels)
ares = ares.fit(X, max_change_num=4, cost_type='MPS', lambda_acc=1.0, lambda_cov=1.0, lambda_cst=1.0)
print('* Parameters:')
print('\t* lambda_acc: {}'.format(ares.lambda_acc)); print('\t* lambda_cov: {}'.format(ares.lambda_cov)); print('\t* lambda_cst: {}'.format(ares.lambda_cst));
print('\t* minimum support: {}'.format(ares.rule_miner_.minsup_)); print('\t* discretization bins: {}'.format(ares.rule_miner_.fd_.bins)); print('\t* pre-processing time[s]: {}'.format(ares.preprocess_time_)); 
print('\t* max rule: {}'.format(ares.max_rule_)); print('\t* max rule length: {}'.format(ares.max_rule_length_)); print('\t* Time[s]:', ares.time_); 
print('\t* uncover test: {}'.format(ares.uncover(X_vl))); print('\t* conflict: {}'.format(ares.conflict(X_vl))); print();
print('### Learned AReS')
print(ares.to_markdown())

## Actionable Recourse Summary
* Parameters:
	* lambda_acc: 1.0
	* lambda_cov: 1.0
	* lambda_cst: 1.0
	* minimum support: 0.05
	* discretization bins: 10
	* pre-processing time[s]: 0.6566287280002143
	* max rule: 4
	* max rule length: 4
	* Time[s]: 234.41160926500015
	* uncover test: 0.08406113537117904
	* conflict: 0.0

### Learned AReS
| | Rule | Action |
| :---: | --- | --- |
| Recourse <br> rule 1 <br> (probability: 49.4%) | If 'juv_fel_count<1' <br> AND 'priors_count<1' | juv_fel_count<1 <br> AND priors_count>=10 |
| Recourse <br> rule 2 <br> (probability: 24.7%) | If 'juv_fel_count<1' <br> AND '1<=priors_count<2' | juv_fel_count<1 <br> AND priors_count>=10 |
| Recourse <br> rule 3 <br> (probability: 11.4%) | If 'juv_fel_count<1' <br> AND '2<=priors_count<3' | juv_fel_count<1 <br> AND priors_count>=10 |
| Recourse <br> rule 4 <br> (probability: 6.6%) | If 'juv_misd_count<1' <br> AND 'juv_other_count<1' <br> AND '4<=priors_count<6' | juv_misd_count<1 <br> AND juv_other_count<1 <br>

In [26]:
x_ares = ares.predict(X_vl) + X_vl
y_ares = mdl.predict(x_ares)

print('Negative accuracy of AReS = {}'.format(np.mean(y_ares != mdl.predict(X_vl))))
print('Negative Inlier AReS = {}'.format(np.mean(isolation.predict(x_ares) == 1)))

Negative accuracy of AReS = 1.0
Negative Inlier AReS = 0.618995633187773


In [28]:
print('Sparsity', np.mean(np.sum(x_ares-X_vl!=0, axis=1)))

Sparsity 1.0


# Counterfactual TREE

In [18]:
print('## Counterfactual Explanation Tree')
cet = CounterfactualExplanationTree(mdl, X_tr, y_tr, max_iteration=500, lime_approximation=(model!='L'),
                                    feature_names=D.feature_names, feature_types=D.feature_types, feature_categories=D.feature_categories,
                                    feature_constraints=D.feature_constraints, target_name=D.target_name, target_labels=D.target_labels)

cet = cet.fit(X, max_change_num=8, cost_type='MPS', C=LAMBDA, gamma=GAMMA, max_leaf_size=-1, time_limit=180, verbose=True)
print('* Parameters:'); print('\t* lambda: {}'.format(cet.lambda_)); print('\t* gamma: {}'.format(cet.gamma_)); print('\t* max_iteration: {}'.format(cet.max_iteration_));
print('\t* leaf size bound:', cet.leaf_size_bound_); print('\t* leaf size:', cet.n_leaves_); print('\t* LIME approximation:', cet.lime_approximation_); print('\t* Time[s]:', cet.time_); print();
print('### Learned CET')
cet.print_tree()

## Counterfactual Explanation Tree
## Stochastic Local Searching ...
### Iteration: 10
#### Before:
- If priors_count<2:
	- If age<29:
		- If race:Caucasian:
			- If priors_count<1:
				- If c_charge_degree:M:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/75 = 0.0% / MeanCost = 0.0705):
						* juv_fel_count: +4
						* juv_misd_count: +13
						* juv_other_count: +6
				- Else:
					* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/103 = 0.0% / MeanCost = 0.0704):
						* juv_misd_count: +10
						* juv_other_count: +7
			- Else:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (45/60 = 75.0% / MeanCost = 0.29):
					* juv_other_count: +5
					* priors_count: +3
		- Else:
			* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (53/443 = 12.0% / MeanCost = 0.0697):
				* juv_fel_count: +6
				* juv_misd_count: +12
				* juv_other_count: +4
	- Else:
		* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidiv

- If race:Native American:
	* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (3/3 = 100.0% / MeanCost = 0.56):
		* juv_misd_count: +13
		* juv_other_count: +9
		* priors_count: +9
- Else:
	- If race:Hispanic:
		- If age<35:
			- If c_charge_degree:F:
				- (+) If gender:
					* (+) Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/47 = 0.0% / MeanCost = 0.0679):
						* juv_other_count: +8
				- Else:
					* (+) Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/14 = 0.0% / MeanCost = 0.0714):
						* juv_misd_count: +8
						* juv_other_count: +9
			- Else:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/58 = 0.0% / MeanCost = 0.0705):
					* juv_fel_count: +4
					* juv_other_count: +8
		- Else:
			* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/155 = 0.0% / MeanCost = 0.0714):
				* juv_misd_count: +7
				* juv_other_count: +9
	- Else:
		* Action [RecidivateWithinTwoYears: Recidivate -> NotRec

- If race:Native American:
	* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (3/3 = 100.0% / MeanCost = 0.56):
		* juv_misd_count: +13
		* juv_other_count: +9
		* priors_count: +9
- Else:
	- If race:Hispanic:
		- If age<96:
			- If c_charge_degree:F:
				- If priors_count<2:
					- If gender:
						- If age<35:
							* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/43 = 0.0% / MeanCost = 0.0676):
								* juv_other_count: +8
						- Else:
							* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (2/44 = 4.5% / MeanCost = 0.467):
								* juv_misd_count: +9
								* juv_other_count: +9
								* priors_count: +5
					- Else:
						- If age<24:
							* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/3 = 0.0% / MeanCost = 0.0711):
								* juv_misd_count: +13
								* juv_other_count: +8
						- Else:
							- If age<45:
								- If age<29:
									* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (

### Iteration: 70
#### Before:
- If race:Native American:
	* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (3/3 = 100.0% / MeanCost = 0.56):
		* juv_misd_count: +13
		* juv_other_count: +9
		* priors_count: +9
- Else:
	- If gender:
		- If age<96:
			- If c_charge_degree:F:
				- If race:Caucasian:
					- If age<35:
						- If priors_count<1:
							* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/113 = 0.0% / MeanCost = 0.0707):
								* juv_fel_count: +2
								* juv_misd_count: +8
								* juv_other_count: +7
						- Else:
							- If age<29:
								* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/33 = 0.0% / MeanCost = 0.0676):
									* juv_fel_count: +1
									* juv_misd_count: +2
									* juv_other_count: +6
							- Else:
								* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/40 = 0.0% / MeanCost = 0.0699):
									* juv_fel_count: +1
									* juv_other_count: +8
					- Else:
						- If prior

- If race:Native American:
	* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (3/3 = 100.0% / MeanCost = 0.56):
		* juv_misd_count: +13
		* juv_other_count: +9
		* priors_count: +9
- Else:
	- If gender:
		- If age<96:
			- If c_charge_degree:F:
				- If race:Caucasian:
					- If age<35:
						- If age<24:
							* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/31 = 0.0% / MeanCost = 0.0632):
								* juv_fel_count: +1
								* juv_other_count: +6
						- Else:
							- If age<29:
								* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/78 = 0.0% / MeanCost = 0.0703):
									* juv_misd_count: +2
									* juv_other_count: +7
							- Else:
								* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/77 = 0.0% / MeanCost = 0.0709):
									* juv_fel_count: +3
									* juv_misd_count: +7
									* juv_other_count: +8
					- Else:
						- If priors_count<1:
							* Action [RecidivateWithinTwoYears: Recidivate -> 

...

- (+) If c_charge_degree:F:
	* (+) Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (1551/1560 = 99.4% / MeanCost = 0.482):
		* juv_misd_count: +3
		* juv_other_count: +6
		* priors_count: +8
- Else:
	- (+) If priors_count<5:
		* (+) Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (1140/1140 = 100.0% / MeanCost = 0.592):
			* juv_misd_count: +1
			* juv_other_count: +4
			* priors_count: +12
	- Else:
		- (+) If race:Other:
			- (+) If age<45:
				* (+) Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/2 = 0.0% / MeanCost = 0.0234):
					* juv_fel_count: +1
			- Else:
				* (+) Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/1 = 0.0% / MeanCost = 0.0372):
					* juv_fel_count: +8
					* priors_count: +1
		- Else:
			- (+) If race:Hispanic:
				* (+) Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/2 = 0.0% / MeanCost = 0.0372):
					* juv_fel_count: +8
					* priors_count: +1
			- Else:
				* (+) Action [Rec

### Iteration: 440
#### Before:
- If c_charge_degree:F:
	* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (1551/1560 = 99.4% / MeanCost = 0.482):
		* juv_misd_count: +3
		* juv_other_count: +6
		* priors_count: +8
- Else:
	- If race:Caucasian:
		- If age<35:
			* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (188/188 = 100.0% / MeanCost = 0.535):
				* juv_fel_count: +1
				* juv_misd_count: +3
				* juv_other_count: +5
				* priors_count: +7
		- Else:
			- If priors_count<1:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (169/189 = 89.4% / MeanCost = 0.646):
					* juv_fel_count: +1
					* juv_other_count: +9
					* priors_count: +8
					* c_charge_degree: "M" -> "F"
			- Else:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (5/155 = 3.2% / MeanCost = 0.107):
					* juv_misd_count: +7
					* juv_other_count: +9
					* priors_count: +1
	- Else:
		* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] 

### Iteration: 470
#### Before:
- If gender:
	* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (2010/2054 = 97.9% / MeanCost = 0.492):
		* juv_misd_count: +3
		* juv_other_count: +6
		* priors_count: +8
- Else:
	- If race:Caucasian:
		- If c_charge_degree:M:
			* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (164/164 = 100.0% / MeanCost = 0.6):
				* juv_misd_count: +1
				* juv_other_count: +5
				* priors_count: +11
		- Else:
			- If priors_count<1:
				* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (63/63 = 100.0% / MeanCost = 0.646):
					* juv_fel_count: +3
					* juv_other_count: +7
					* priors_count: +8
			- Else:
				- If age<29:
					- If age<24:
						* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (0/4 = 0.0% / MeanCost = 0.0701):
							* juv_fel_count: +8
							* juv_misd_count: +5
							* juv_other_count: +4
					- Else:
						- If priors_count<2:
							* Action [RecidivateWithinTwoYears: Recidivate 

### Iteration: 490
#### Before:
- If gender:
	* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (2010/2054 = 97.9% / MeanCost = 0.492):
		* juv_misd_count: +3
		* juv_other_count: +6
		* priors_count: +8
- Else:
	- If c_charge_degree:M:
		* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (328/328 = 100.0% / MeanCost = 0.613):
			* juv_other_count: +4
			* priors_count: +12
	- Else:
		- If priors_count<1:
			* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (164/164 = 100.0% / MeanCost = 0.646):
				* juv_fel_count: +3
				* juv_other_count: +7
				* priors_count: +8
		- Else:
			- If age<29:
				- If age<24:
					- If priors_count<2:
						* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (1/12 = 8.3% / MeanCost = 0.0709):
							* juv_other_count: +6
					- Else:
						* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (2/2 = 100.0% / MeanCost = 0.0504):
							* juv_fel_count: +1
							* juv_other_count: +1
		

- If gender:
	* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (2010/2054 = 97.9% / MeanCost = 0.492):
		* juv_misd_count: +3
		* juv_other_count: +6
		* priors_count: +8
- Else:
	- If c_charge_degree:M:
		* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (328/328 = 100.0% / MeanCost = 0.613):
			* juv_other_count: +4
			* priors_count: +12
	- Else:
		- If priors_count<1:
			* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (164/164 = 100.0% / MeanCost = 0.646):
				* juv_fel_count: +3
				* juv_other_count: +7
				* priors_count: +8
		- Else:
			- If age<29:
				- If age<24:
					- If priors_count<2:
						* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (1/12 = 8.3% / MeanCost = 0.0709):
							* juv_other_count: +6
					- Else:
						* Action [RecidivateWithinTwoYears: Recidivate -> NotRecidivate] (2/2 = 100.0% / MeanCost = 0.0504):
							* juv_fel_count: +1
							* juv_other_count: +1
				- Else:
					* Action [Recidiv

In [29]:
x_cet = cet.predict(X_vl) + X_vl
y_cet = mdl.predict(x_cet)

print('Negative accuracy of CET = {}'.format(np.mean(y_cet != mdl.predict(X_vl))))
print('Negative Oulier CET = {}'.format(np.mean(isolation.predict(x_cet) == -1)))

Negative accuracy of CET = 0.9847161572052402
Negative Oulier CET = 1.0


In [30]:
print('Sparsity', np.mean(np.sum(x_cet-X_vl!=0, axis=1)))

Sparsity 2.4716157205240177
